<a href="https://colab.research.google.com/github/iamjimmycai/five_star/blob/main/tf_classification_tricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import math
K = tf.keras.backend



tf.random.set_seed(42)


In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

# Batch Normalization


In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Flatten(input_shape=[28, 28]),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dense(300, activation="relu",
#                           kernel_initializer="he_normal"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dense(100, activation="relu",
#                           kernel_initializer="he_normal"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dense(10, activation="softmax")
# ])

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# model.summary()

# tf.keras.utils.plot_model(model, show_shapes=True)

# [(var.name, var.trainable) for var in model.layers[1].variables]



In [ ]:
# # extra code – just show that the model works! 😊
# model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",
#               metrics=["accuracy"])
# model.fit(X_train, y_train, epochs=2, validation_data=(X_valid, y_valid))

optimizer = tf.keras.optimizers.SGD(clipvalue=1.0)
# optimizer = tf.keras.optimizers.SGD(clipnorm=1.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(X_train, y_train, epochs=2, validation_data=(X_valid, y_valid))


Epoch 1/2
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8887 - loss: 0.3152 - val_accuracy: 0.8706 - val_loss: 0.3515
Epoch 2/2
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8970 - loss: 0.2918 - val_accuracy: 0.8760 - val_loss: 0.3485


# Reusing Pretrained Layers

In [ ]:
# extra code – split Fashion MNIST into tasks A and B, then train and save
#              model A to "my_model_A".

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
pos_class_id = class_names.index("Pullover")
neg_class_id = class_names.index("T-shirt/top")

def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [ ]:
# X_train
# X_train_A, y_train_A
# y_train_B, y_train_A

In [ ]:
model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4843 - loss: 1.5306 - val_accuracy: 0.7907 - val_loss: 0.6935
Epoch 2/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8026 - loss: 0.6452 - val_accuracy: 0.8353 - val_loss: 0.5117
Epoch 3/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8438 - loss: 0.4986 - val_accuracy: 0.8611 - val_loss: 0.4347
Epoch 4/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8648 - loss: 0.4275 - val_accuracy: 0.8704 - val_loss: 0.3901
Epoch 5/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8736 - loss: 0.3851 - val_accuracy: 0.8764 - val_loss: 0.3627
Epoch 6/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8807 - loss: 0.3579 - val_accuracy: 0.8819 - val_loss: 0.3440
Epoch 7/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8854 - loss: 0.3387 - val_accuracy: 0.8834 - val_loss: 0.3303
Epoch 8/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8893 - loss: 0.3240 - 

In [ ]:
model_A.save("my_model_A.keras")

In [ ]:
model_B = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model_B.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - accuracy: 0.5301 - loss: 0.7520 - val_accuracy: 0.4955 - val_loss: 0.7535
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5328 - loss: 0.7268 - val_accuracy: 0.4985 - val_loss: 0.7306
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5535 - loss: 0.7063 - val_accuracy: 0.5094 - val_loss: 0.7109
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5611 - loss: 0.6884 - val_accuracy: 0.5223 - val_loss: 0.6931
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5753 - loss: 0.6718 - val_accuracy: 0.5579 - val_loss: 0.6769
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6162 - loss: 0.6564 - val_accuracy: 0.5925 - val_loss: 0.6619
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6924 - loss: 0.6419 - val_accuracy: 0.6558 - val_loss: 0.6479
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7616 - loss: 0.6283 - val_accuracy: 0.7270 - val_loss: 0.6350

In [ ]:
model_B.evaluate(X_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9102 - loss: 0.5311


[0.5309159755706787, 0.9100000262260437]

In [ ]:
model_A = tf.keras.models.load_model("my_model_A.keras")
model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

# model_A_clone = tf.keras.models.clone_model(model_A)
# model_A_clone.set_weights(model_A.get_weights())
# # extra code – creating model_B_on_A just like in the previous cell
# model_B_on_A = tf.keras.Sequential(model_A_clone.layers[:-1])
# model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

# 前期先冻结，训练几个epoch
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.4301 - loss: 1.4848 - val_accuracy: 0.4847 - val_loss: 0.9781
Epoch 2/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4404 - loss: 0.9040 - val_accuracy: 0.5321 - val_loss: 0.7266
Epoch 3/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5460 - loss: 0.6865 - val_accuracy: 0.6004 - val_loss: 0.6621
Epoch 4/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5973 - loss: 0.6282 - val_accuracy: 0.6400 - val_loss: 0.6400


In [ ]:
# 后期不冻结，微调几十个epoch
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.6832 - loss: 0.5934 - val_accuracy: 0.7161 - val_loss: 0.5877
Epoch 2/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7477 - loss: 0.5345 - val_accuracy: 0.7923 - val_loss: 0.5359
Epoch 3/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8240 - loss: 0.4807 - val_accuracy: 0.8417 - val_loss: 0.4938
Epoch 4/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8840 - loss: 0.4384 - val_accuracy: 0.8684 - val_loss: 0.4600
Epoch 5/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8968 - loss: 0.4047 - val_accuracy: 0.8783 - val_loss: 0.4326
Epoch 6/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9361 - loss: 0.3772 - val_accuracy: 0.8932 - val_loss: 0.4095
Epoch 7/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9461 - loss: 0.3543 - val_accuracy: 0.8991 - val_loss: 0.3899
Epoch 8/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9497 - loss: 0.3347 - val_accuracy: 0.9050 - val_loss: 0.3728


In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9218 - loss: 0.2861


[0.29569366574287415, 0.9190000295639038]

# Learning Rate Scheduling


In [ ]:
# extra code – a little function to test an optimizer on Fashion MNIST

def build_model(seed=42):
    tf.random.set_seed(seed)
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=[28, 28]),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

def build_and_train_model(optimizer):
    model = build_model()
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model.fit(X_train, y_train, epochs=5,
                     validation_data=(X_valid, y_valid))

In [ ]:
# DEPRECATED:
# optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, decay=1e-4)

# RECOMMENDED:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate=0.01,
    decay_steps=10_000,
    decay_rate=1.0,
    staircase=False
)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
history_power_scheduling = build_and_train_model(optimizer)  # extra code

Epoch 1/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6858 - loss: 0.9475 - val_accuracy: 0.8336 - val_loss: 0.4887
Epoch 2/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8275 - loss: 0.4925 - val_accuracy: 0.8446 - val_loss: 0.4513
Epoch 3/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8460 - loss: 0.4422 - val_accuracy: 0.8486 - val_loss: 0.4324
Epoch 4/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8550 - loss: 0.4142 - val_accuracy: 0.8524 - val_loss: 0.4186
Epoch 5/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8620 - loss: 0.3946 - val_accuracy: 0.8540 - val_loss: 0.4068


In [ ]:
# def exponential_decay_fn(epoch):
#     return 0.01 * 0.1 ** (epoch / 20)

# def exponential_decay_fn(epoch, lr):
#     return lr * 0.1 ** (1 / 20)

def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 ** (epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)
model = build_model()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid),
                    callbacks=[lr_scheduler])

Epoch 1/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6820 - loss: 0.9507 - val_accuracy: 0.8318 - val_loss: 0.4784 - learning_rate: 0.0100
Epoch 2/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8300 - loss: 0.4819 - val_accuracy: 0.8382 - val_loss: 0.4387 - learning_rate: 0.0089
Epoch 3/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8480 - loss: 0.4315 - val_accuracy: 0.8420 - val_loss: 0.4195 - learning_rate: 0.0079
Epoch 4/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8578 - loss: 0.4043 - val_accuracy: 0.8486 - val_loss: 0.4061 - learning_rate: 0.0071
Epoch 5/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8629 - loss: 0.3857 - val_accuracy: 0.8544 - val_loss: 0.3964 - learning_rate: 0.0063


# 1Cycle scheduling


In [ ]:
class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(self, iterations, max_lr=1e-3, start_lr=None,
                 last_iterations=None, last_lr=None):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr,
                                   self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                   self.max_lr, self.start_lr)
        else:
            lr = self._interpolate(2 * self.half_iteration, self.iterations,
                                   self.start_lr, self.last_lr)
        self.iteration += 1
        # K.set_value(self.model.optimizer.learning_rate, lr)
        self.model.optimizer.learning_rate.assign(lr)

model = build_model()
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(),
              metrics=["accuracy"])
n_epochs = 5
batch_size = 128
onecycle = OneCycleScheduler(math.ceil(len(X_train) / batch_size) * n_epochs,
                             max_lr=0.1)
history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size,
                    validation_data=(X_valid, y_valid),
                    callbacks=[onecycle])

Epoch 1/5
430/430 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5917 - loss: 1.2481 - val_accuracy: 0.7704 - val_loss: 0.6456
Epoch 2/5
430/430 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7958 - loss: 0.5767 - val_accuracy: 0.7902 - val_loss: 0.5681
Epoch 3/5
430/430 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8273 - loss: 0.4785 - val_accuracy: 0.8326 - val_loss: 0.4521
Epoch 4/5
430/430 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8593 - loss: 0.3860 - val_accuracy: 0.8580 - val_loss: 0.3818
Epoch 5/5
430/430 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8736 - loss: 0.3464 - val_accuracy: 0.8680 - val_loss: 0.3533


# Avoiding Overfitting Through Regularization

In [ ]:
# layer = tf.keras.layers.Dense(100, activation="relu",
#                               kernel_initializer="he_normal",
#                               kernel_regularizer=tf.keras.regularizers.l2(0.01))

from functools import partial

RegularizedDense = partial(tf.keras.layers.Dense,
                           activation="relu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(100),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
optimizer = tf.keras.optimizers.SGD(learning_rate=0.02)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=2,
                    validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7002 - loss: 4.1179 - val_accuracy: 0.8252 - val_loss: 1.8620
Epoch 2/2
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8125 - loss: 1.6270 - val_accuracy: 0.8244 - val_loss: 1.1175
